In [ ]:
# -*- coding: utf-8 -*-
"""
IED por Entidad Federativa (últimos 4 trimestres)
- Fuente: Secretaría de Economía (datos.gob.mx / CKAN)
- Salida: Figura Plotly con estilo corporativo y estado resaltado + Streamlit Chart
"""
import streamlit as st
import io
import json
import math
import unicodedata
from datetime import datetime
from typing import Tuple

import pandas as pd
import requests
import plotly.graph_objects as go
import sys

# ==========================
# Configuración de estilo (Se usará el contexto de Streamlit)
# ==========================
# Valores por defecto para ejecución fuera de Streamlit
DEFAULT_FONT = "Arial, sans-serif"
DEFAULT_BASE = "#0b132b"
DEFAULT_HIGHLIGHT = "#889064"
BG_COLOR = "white"
GRID_COLOR = "#d9d9d9"

# Variables globales que serán actualizadas por Streamlit o usarán el default
FONT_PRIMARY = DEFAULT_FONT
COLOR_BASE = DEFAULT_BASE
COLOR_HIGHLIGHT = DEFAULT_HIGHLIGHT
COLOR_ACCENT = "#ff9f18"

FIG_WIDTH = 1200
FIG_HEIGHT = 460
TITLE = "Flujo de inversión extranjera"
SUBTITLE = "(Últimos 4 trimestres, Millones USD)"


# ==========================
# Utilidades
# ==========================
def normalize(s: str) -> str:
    """Quita acentos/diacríticos y pasa a minúsculas para comparaciones robustas."""
    if s is None:
        return ""
    s = s.strip()
    s = "".join(
        c for c in unicodedata.normalize("NFD", s)
        if unicodedata.category(c) != "Mn"
    )
    return s.lower()


def map_estado_alias(estado: str) -> str:
    """Normaliza aliases comunes (CDMX, EdoMex, etc.) a nombre oficial."""
    s = normalize(estado)
    aliases = {
        "cdmx": "Ciudad de México",
        "ciudad de mexico": "Ciudad de México",
        "distrito federal": "Ciudad de México",
        "edomex": "Estado de México",
        "mexico": "Estado de México",
        "coahuila de zaragoza": "Coahuila",
        "veracruz de ignacio de la llave": "Veracruz",
        "michoacan de ocampo": "Michoacán",
        "quintana roo": "Quintana Roo",
        "yucatan": "Yucatán",
        "nuevo leon": "Nuevo León",
        "san luis potosi": "San Luis Potosí",
        "baja california": "Baja California",
        "baja california sur": "Baja California Sur",
    }
    return aliases.get(s, estado.strip())


def ckan_package_show(dataset_id: str) -> dict:
    """Consulta CKAN package_show de datos.gob.mx y regresa el JSON."""
    url = f"https://www.datos.gob.mx/api/3/action/package_show?id={dataset_id}"
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    data = r.json()
    if not data.get("success"):
        raise RuntimeError("CKAN devolvió success=False")
    return data["result"]


def pick_resource_url(resources: list) -> str:
    """Selecciona el recurso CSV 'por entidad' con granularidad trimestral (o el más cercano)."""
    candidates = []
    for res in resources:
        fmt = (res.get("format") or "").lower()
        name = (res.get("name") or "") + " " + (res.get("description") or "")
        name_l = normalize(name)
        if fmt != "csv":
            continue
        score = 0
        if "entidad" in name_l:
            score += 2
        if "trimestre" in name_l or "trimestral" in name_l:
            score += 2
        if "tipo" in name_l:
            score += 1
        if score > 0:
            updated = res.get("last_modified") or res.get("revision_timestamp") or ""
            candidates.append((score, updated, res.get("url")))

    if not candidates:
        for res in resources:
            fmt = (res.get("format") or "").lower()
            name = (res.get("name") or "") + " " + (res.get("description") or "")
            name_l = normalize(name)
            if fmt == "csv" and "entidad" in name_l:
                updated = res.get("last_modified") or res.get("revision_timestamp") or ""
                candidates.append((1, updated, res.get("url")))

    if not candidates:
        raise RuntimeError("No encontré un recurso CSV adecuado 'por entidad' en el dataset.")

    # Ordenar por score desc, luego por fecha de actualización desc (ISO)
    candidates.sort(key=lambda x: (x[0], x[1]), reverse=True)
    return candidates[0][2]


def read_csv_robusto(url: str) -> pd.DataFrame:
    """Lee CSV tratando codificaciones comunes y separadores estándar."""
    try:
        return pd.read_csv(url)
    except Exception:
        pass
    try:
        return pd.read_csv(url, encoding="latin-1")
    except Exception:
        pass
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return pd.read_csv(io.BytesIO(r.content))


def detectar_columnas(df: pd.DataFrame) -> Tuple[str, str, str, str]:
    """Detecta nombres de columnas: entidad, anio, trimestre, valor."""
    cols = {c.lower(): c for c in df.columns}

    col_entidad = next((cols[c] for c in cols if "entidad" in c and "federativa" in c), None)
    if col_entidad is None:
        col_entidad = next((cols[c] for c in cols if "entidad" in c), None)
    if col_entidad is None:
        col_entidad = next((cols[c] for c in cols if "estado" in c), None)

    col_anio = next((cols[c] for c in cols if c in ("anio", "año", "year")), None)
    col_trim = next((cols[c] for c in cols if "trimestre" in c or c in ("trim", "quarter")), None)

    # valor: busca columnas que contengan usd/mdd/millones/valor/monto
    col_valor = None
    for key in ["usd", "mdd", "miles de dolares", "millones", "valor", "monto"]:
        col_valor = next((cols[c] for c in cols if key in c), None)
        if col_valor:
            break

    if col_valor is None:
        numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
        for c in [col_anio, col_trim]:
            if c in numeric_cols:
                numeric_cols.remove(c)
        col_valor = numeric_cols[0] if numeric_cols else None

    if not all([col_entidad, col_anio, col_trim, col_valor]):
        raise RuntimeError(f"No pude identificar columnas clave. Columnas disponibles: {list(df.columns)}")

    return col_entidad, col_anio, col_trim, col_valor


def rolling_4t(
    df: pd.DataFrame,
    col_entidad: str,
    col_anio: str,
    col_trim: str,
    col_valor: str
) -> Tuple[pd.DataFrame, str]:
    """Suma de los últimos 4 trimestres por entidad, usando la observación más reciente disponible."""
    df = df.copy()
    df[col_trim] = df[col_trim].astype(str).str.extract(r"(\d+)").astype(int)
    df[col_anio] = df[col_anio].astype(int)

    # Chequeo de datos mínimos
    if df.empty or df[col_anio].max() is None or df[col_trim].max() is None:
        return pd.DataFrame(columns=[col_entidad, "mdd_4t", "rank"]), ""

    max_year = df[col_anio].max()
    max_q = df.loc[df[col_anio] == max_year, col_trim].max()

    df["_t"] = df[col_anio] * 4 + df[col_trim]
    t_max = max_year * 4 + max_q
    window = {t_max, t_max - 1, t_max - 2, t_max - 3}

    df_4t = df[df["_t"].isin(window)].copy()
    g = df_4t.groupby(col_entidad, as_index=False)[col_valor].sum()
    g.rename(columns={col_valor: "mdd_4t"}, inplace=True)
    g.sort_values("mdd_4t", ascending=False, inplace=True)
    g["rank"] = range(1, len(g) + 1)

    periodo = f"{max_year}T{max_q}"
    return g, periodo


def formatea_mdd(x: float) -> str:
    """Formatea millones de USD con separador de miles y sin decimales."""
    return f"${abs(x):,.0f} M USD"


def escala_y_publicacion(valores):
    """Devuelve (y_min, y_max, tick0, dtick) con estética tipo publicación."""
    vmin = float(min(valores))
    vmax = float(max(valores))

    y_min = -500 if vmin < 0 else 0

    def round_up_500(x):
        return int((math.ceil(x / 500.0)) * 500)

    y_max = round_up_500(vmax) + 500

    tick0 = y_min
    dtick = 1000
    return y_min, y_max, tick0, dtick


def construir_figura(df_rank: pd.DataFrame, estado_sel: str, periodo: str) -> go.Figure:
    # Usar variables globales de estilo, ya actualizadas en main()
    global FONT_PRIMARY, COLOR_BASE, COLOR_HIGHLIGHT, BG_COLOR, GRID_COLOR, TITLE, SUBTITLE, FIG_WIDTH, FIG_HEIGHT

    estados = df_rank.iloc[:, 0].tolist()
    valores = df_rank["mdd_4t"].tolist()

    # Colores y match del estado seleccionado
    colors = [COLOR_BASE] * len(estados)
    estado_norm = normalize(estado_sel)

    match_idx = None
    for i, e in enumerate(estados):
        if normalize(e) == estado_norm:
            match_idx = i
            break
    if match_idx is None:
        estado_mapped = map_estado_alias(estado_sel)
        estado_norm2 = normalize(estado_mapped)
        for i, e in enumerate(estados):
            if normalize(e) == estado_norm2:
                match_idx = i
                estado_sel = estado_mapped
                break

    if match_idx is not None:
        colors[match_idx] = COLOR_HIGHLIGHT

    # Figura base
    fig = go.Figure()
    fig.add_bar(
        x=estados,
        y=valores,
        marker_color=colors,
        hovertemplate="<b>%{x}</b><br>%{y:,.0f} M USD<extra></extra>",
    )

    # === Escala Y tipo publicación (–500/0 base y múltiplos de 500) ===
    y_min, y_max, tick0, dtick = escala_y_publicacion(valores)

    fig.update_layout(
        width=FIG_WIDTH,
        height=FIG_HEIGHT,
        plot_bgcolor=BG_COLOR,
        paper_bgcolor=BG_COLOR,
        font=dict(family=FONT_PRIMARY, size=16, color="#111"),
        # === CORRECCIÓN 1: Aumentamos el margen inferior (b) a 200 ===
        # Esto crea espacio físico suficiente para que el texto no se corte
        margin=dict(l=60, r=40, t=70, b=200),
        title=dict(
            text=f"{TITLE}<br><span style='font-size:14px;color:#666;'>{SUBTITLE} · {periodo}</span>",
            x=0.5, xanchor="center",
            font=dict(family=FONT_PRIMARY)
        ),
        xaxis=dict(
            tickangle=90,
            showgrid=False,
            tickfont=dict(size=12, family=FONT_PRIMARY),
        ),
        yaxis=dict(
            title=None,
            showgrid=True,
            gridcolor=GRID_COLOR,
            zeroline=True,
            zerolinecolor="#bbb",
            tickformat=",.0f",
            range=[y_min, y_max],
            tick0=tick0,
            dtick=dtick,
            tickfont=dict(family=FONT_PRIMARY)
        ),
    )

    # === Recuadro SIEMPRE en la esquina superior derecha ===
    if match_idx is not None:
        estado_box = estados[match_idx]
        valor_box = valores[match_idx]
        color_box = COLOR_HIGHLIGHT
    else:
        estado_box = estados[0]
        valor_box = valores[0]
        color_box = COLOR_BASE

    texto = f"<b>{estado_box}</b><br>{formatea_mdd(valor_box)}"

    # 1) Anotación (Box)
    fig.add_annotation(
        x=0.92, y=0.93, xref="paper", yref="paper",
        text=texto,
        showarrow=False,
        align="center",
        font=dict(color="#444", size=14, family=FONT_PRIMARY),
        bordercolor="rgba(0,0,0,0)",
        borderwidth=0,
        borderpad=8,
        bgcolor="rgba(255,255,255,0.25)",
        opacity=0.98
    )

    # 2) Marco punteado
    fig.add_shape(
        type="rect",
        xref="paper", yref="paper",
        x0=0.78, x1=0.98, y0=0.74, y1=0.92,
        line=dict(color=color_box, width=1, dash="dot"),
        fillcolor="rgba(0,0,0,0)",
        layer="above"
    )

    # === CORRECCIÓN 2: Leyenda en parte Inferior Derecha ===
    fig.add_annotation(
        text="Fuente: Secretaría de Economía (datos.gob.mx)",
        xref="paper", yref="paper",
        # x=1 coloca el texto pegado al borde derecho
        x=0.01, 
        # y=-0.45 lo baja lo suficiente para no chocar con los estados, 
        # pero gracias al margin-bottom=200, sigue siendo visible.
        y=-0.95, 
        showarrow=False,
        xanchor='left',  # Alineación a la derecha
        yanchor='bottom', # Alineación al fondo
        font=dict(size=11, color="gray", family=FONT_PRIMARY)
    )

    return fig


def main():
    # Acceder a las variables del contexto de Streamlit inyectadas por exec()
    global FONT_PRIMARY, COLOR_BASE, COLOR_HIGHLIGHT

    # 1. Configurar estilos con el contexto de Streamlit
    try:
        FONT_PRIMARY = active_font
        COLOR_HIGHLIGHT = active_palette[0]
        if len(active_palette) > 1:
            COLOR_BASE = active_palette[1]
        else:
            COLOR_BASE = DEFAULT_BASE # Fallback original

    except NameError:
        pass # Los defaults se mantienen


    # 2) Estado por teclado (simulado)
    estado_input = None
    try:
        # Llama a input() sin argumentos para suprimir la impresión del prompt.
        estado_input = input().strip()
    except Exception:
        pass

    if not estado_input:
        estado_input = "Aguascalientes"
        # Este st.info o print() evita el warning de 'no hubo salida en la consola'
        # en app.py si se usa el default, sin mostrar el prompt de input.
        st.info(f"Usando '{estado_input}' como valor por defecto para el Estado a resaltar.")


    # 3) Dataset en datos.gob.mx (CKAN)
    DATASET_ID = "inversion_extranjera_directa"
    resource_url = ""
    try:
        # 4) Obtención de datos y procesamiento
        pkg = ckan_package_show(DATASET_ID)
        resource_url = pick_resource_url(pkg["resources"])

        df_raw = read_csv_robusto(resource_url)

        col_entidad, col_anio, col_trim, col_valor = detectar_columnas(df_raw)
        df = df_raw[[col_entidad, col_anio, col_trim, col_valor]].copy()
        df[col_valor] = pd.to_numeric(df[col_valor], errors="coerce")
        df = df.dropna(subset=[col_valor])

        df_rank, periodo = rolling_4t(df, col_entidad, col_anio, col_trim, col_valor)

        # CHEQUEO DE DATOS ANTES DE CONSTRUIR LA FIGURA
        if df_rank.empty:
            st.error("❌ El dataset fue cargado, pero **no contiene datos válidos** para calcular la IED de los últimos 4 trimestres.")
            st.caption(f"Verifica que el recurso en {resource_url} contenga datos trimestrales recientes.")
        else:
            # 6) Figura
            fig = construir_figura(df_rank, map_estado_alias(estado_input), periodo)

            # 7) Mostrar en Streamlit
            st.plotly_chart(fig, use_container_width=True)

            # 8) Mostrar Tabla de Datos (SIN EXPANDERS)
            st.markdown("**Datos detallados por Entidad Federativa**")
            
            # Preparamos el DF para mostrar: Renombrar y ordenar
            df_table = df_rank.copy()
            # Asumiendo que la columna 0 es la entidad, y "mdd_4t" el valor, "rank" la posición
            col_ent_name = df_table.columns[0]
            
            df_table = df_table.rename(columns={
                col_ent_name: "Entidad Federativa",
                "mdd_4t": "Monto (Millones USD)",
                "rank": "Posición Nacional"
            })
            
            # Reordenar columnas para que "Posición" salga primero (opcional, pero se ve mejor)
            df_table = df_table[["Posición Nacional", "Entidad Federativa", "Monto (Millones USD)"]]

            st.dataframe(
                df_table,
                use_container_width=True,
                hide_index=True,
                column_config={
                    "Posición Nacional": st.column_config.NumberColumn(format="%d"),
                    "Monto (Millones USD)": st.column_config.NumberColumn(format="$%,.1f M")
                }
            )

    except requests.exceptions.HTTPError as he:
        st.error(f"Error HTTP al descargar datos: Asegúrate de que la URL de datos.gob.mx esté activa. ({he})")
        st.code(f"URL de recurso fallida: {resource_url}")
    except RuntimeError as re:
        st.error(f"Error en el procesamiento de datos: {re}")
    except Exception as e:
        # Muestra el error genérico
        st.error(f"Ocurrió un error inesperado al generar la gráfica. Intente otro estado. Detalle: {e}")

if __name__ == "__main__":
    main()

Escribe el nombre del Estado a resaltar (p. ej., 'Hidalgo', 'CDMX', 'Estado de México'): Hidalgo


2025-11-30 23:43:37.935 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2025-11-30 23:43:37.937 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-30 23:43:37.938 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-30 23:43:37.939 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-30 23:43:38.007 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-11-30 23:43:38.008 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-30 23:43:38.009 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare 